<a href="https://colab.research.google.com/github/AbdulMoiz-MMY/Agentic_AI_Learning/blob/main/GENAI_02_Onsite_Assignment(Rag_Prac)1_1_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

from pinecone import Pinecone,ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

if not pinecone_api_key:
    raise ValueError("PINECONE_API_KEY not set")

pc = Pinecone(api_key = pinecone_api_key)

In [3]:
index_name = "onclass-rag-project"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension= 768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
vector = embeddings.embed_query("We are building a rag query")

In [6]:
vector[:5]

[0.03296078369021416,
 -0.05385661870241165,
 -0.0600518062710762,
 -0.032095782458782196,
 -0.009051025845110416]

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index,embeddings)

In [8]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.2 MB/s eta 0:00:00


In [9]:
!pip install PyPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.7 MB/s eta 0:00:00


In [11]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/personal_finance_advisor.pdf")
data = loader.load()

In [12]:
len(data)

22

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

docs = text_splitter.split_documents(data)

len(docs)

66

In [14]:
docs[3]

Document(metadata={'source': '/content/personal_finance_advisor.pdf', 'page': 1}, page_content='How can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Creat

In [15]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs,ids=uuids)

['ddd9f5e1-6e3b-44db-91c8-73c242fd8296',
 '5885576a-72a6-4520-8941-7ad9167e83a3',
 'd4d79a6b-ae08-44e8-ae70-a0302fb8f7e0',
 '0a20adb3-4b33-4546-b766-40343bdf598e',
 'ca68cf2c-297d-4687-8ae8-2fedabecfead',
 '58184898-58e1-4fb3-9431-e51a0593335c',
 'c1d4a796-e776-404a-bf2f-99db621818ea',
 '87252ba7-b257-422c-9e97-646a4caeaac6',
 'bfca8bc4-fac5-4e66-8e48-22bdc575163f',
 '4fe73b0a-9e87-484e-9c31-513de220e2fd',
 '590af3f8-2f07-4c18-b675-21b8746f1848',
 'c919d3a8-aa9f-49b6-8bee-f761d63ed125',
 '9d94531c-4344-4d55-910c-07188af446ec',
 '7e5d8611-9fa6-4577-abb6-aa425ec7319c',
 '81c2145e-2bc3-4e75-9cc7-206067c92928',
 '4c0e0083-f8c6-46fc-be03-cc0f82590b09',
 '529d97f6-00d7-4d99-822a-0f70cb344ccd',
 '12051719-8202-4868-aa85-a836b5edc5f9',
 '9e133ca6-e535-4b80-a290-015b752163f4',
 '9dfc26de-0efb-4733-bd21-779dec0d3f20',
 '6bcb891f-6499-449a-826d-06de7f3e2ab8',
 '60f1d9c0-cd22-49ce-b3d1-79f5fc68d3ec',
 'c100de9f-d7d3-4388-923e-3e319dff03cb',
 '6f6ffa9f-8e84-4b3f-a64a-bff441fd88c5',
 'c9109b22-83b3-

In [16]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("Return Policy for Product 452 and 950")

In [17]:
retrieved_docs

[]

In [18]:
len(retrieved_docs)

0

In [34]:
# print(retrieved_docs[5].page_content)
if retrieved_docs:
    print(retrieved_docs[4].page_content)
else:
    print("retrieved_docs is empty, cannot access element at index 5.")

retrieved_docs is empty, cannot access element at index 5.


In [20]:
results = vector_store.similarity_search(
    "Return Policy for Product 9,50",
    k=10
)

for res in results :
  print(f"* {res.page_content} [{res.metadata}]")

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash"
)

In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the followingpieces of retrieved context to answer the question"
    "If you dont know the answer, say that you don't know. Use three sentences maximum and keep answers consise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)



In [23]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)

In [24]:
from IPython.display import display,Markdown

response = rag_chain.invoke({"input":"Return Policy for product 625"})
print("Input:", response['input'])
print(f"Context: {response['context'][:2]}")
print("Answer:", response['answer'])

Input: Return Policy for product 625
Context: [Document(id='ddd9f5e1-6e3b-44db-91c8-73c242fd8296', metadata={'page': 0.0, 'source': '/content/personal_finance_advisor.pdf'}, page_content='Query Advice\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on grocer

In [25]:
response = rag_chain.invoke({"input":"Return Policy for product 452 and 950"})
print("Input:", response['input'])
print(f"Context: {response['context'][:2]}")
print("Answer:", response['answer'])

Input: Return Policy for product 452 and 950
Context: [Document(id='ddd9f5e1-6e3b-44db-91c8-73c242fd8296', metadata={'page': 0.0, 'source': '/content/personal_finance_advisor.pdf'}, page_content='Query Advice\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money o

In [26]:
def answer_to_query(query):
    vector_results = vector_store.similarity_search(query, k=10)
    response = llm.invoke(
        f"Answer this query: {query}\nHere are some references: {vector_results}"
    )

    if hasattr(response, 'content'):
        ans = response.content
    else:
        ans = response

    return ans, query, vector_results

In [27]:
ans, query, content = answer_to_query("For Product 652. What is the return policy?")

print(f"Input : {query}")
print(f"Content : {content}")
print(f"Answer : {ans}")

Input : For Product 652. What is the return policy?
Content : [Document(id='ddd9f5e1-6e3b-44db-91c8-73c242fd8296', metadata={'page': 0.0, 'source': '/content/personal_finance_advisor.pdf'}, page_content='Query Advice\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save money on groceries? Create a budget\nHow can I save money on rent? Create a budget\nHow can I save money on utilities? Create a budget\nHow can I save money on entertainment? Create a budget\nHow can I save money on travel? Create a budget\nHow can I save